# Regularization Mode

## Theory

## Example

In [1]:
from saenopy import Solver
    
# initialize the object
M = Solver()

Set the material model:

In [2]:
from saenopy.materials import SemiAffineFiberMaterial

# provide a material model
material = SemiAffineFiberMaterial(1645, 0.0008, 0.0075, 0.033)
M.setMaterialModel(material)

Load the mesh from files.

In [3]:
import numpy as np

# define the coordinates of the nodes of the mesh
# the array has to have the shape N_v x 3
R = np.array([[0., 0., 0.],  # 0
              [0., 1., 0.],  # 1
              [1., 1., 0.],  # 2
              [1., 0., 0.],  # 3
              [0., 0., 1.],  # 4
              [1., 0., 1.],  # 5
              [1., 1., 1.],  # 6
              [0., 1., 1.]]) # 7

# define the tetrahedra of the mesh
# the array has to have the shape N_t x 4
# every entry is an index referencing a verces in R (indices start with 0)
T = np.array([[0, 1, 7, 2],
              [0, 2, 5, 3],
              [0, 4, 5, 7],
              [2, 5, 6, 7],
              [0, 7, 5, 2]])

And hand the data over to the FiniteBodyForces object.

In [4]:
# provide the node data
M.setNodes(R)
# and the tetrahedron data
M.setTetrahedra(T)

Now we have to specify which displacements to fit.

In [5]:
# the displacements of the nodes which shall be fitted
# during the solving
U = np.array([[0   , 0, 0],  # 0
              [0   , 0, 0],  # 1
              [0.01, 0, 0],  # 2
              [0.01, 0, 0],  # 3
              [0   , 0, 0],  # 4
              [0.01, 0, 0],  # 5
              [0.01, 0, 0],  # 6
              [0   , 0, 0]]) # 7

# hand the displacements over to the class instance
M.setTargetDisplacements(U)

Now we can start the regularisation process.

In [6]:
# relax the mesh and move the "varible" nodes
M.regularize(stepper=0.1, alpha=0.001)

going to update glo f and K
updating forces and stiffness matrix finished 3.72s
|u-uf|^2 = 0.0004 		perbead= 0.005
|w*f|^2  = 4.146380843321553e-28 		|u|^2 = 0.0
L = |u-uf|^2 + lambda*|w*f|^2 =  0.0004
check before relax !
total weight:  0.1366493918715871 / 8
updating forces and stiffness matrix finished 0.00s
Round 1  |du|= 0.0005213814018435404
|u-uf|^2 = 0.0003531271885103128 		perbead= 0.004890180659032298
|w*f|^2  = 0.0011037073817204528 		|u|^2 = 2.1747085295066833e-06
L = |u-uf|^2 + lambda*|w*f|^2 =  0.0003542308958920332
total weight:  0.0 / 8
updating forces and stiffness matrix finished 0.00s
Round 2  |du|= 0.0004680788598684218
|u-uf|^2 = 0.0003151902582904738 		perbead= 0.004798534891071429
|w*f|^2  = 0.0036773637113855376 		|u|^2 = 7.831754829752018e-06
L = |u-uf|^2 + lambda*|w*f|^2 =  0.00031886762200185933
total weight:  0.0 / 8
updating forces and stiffness matrix finished 0.00s
Round 3  |du|= 0.00042044002573673697
|u-uf|^2 = 0.00028440968709130996 		perbead= 0.004721

[(0.0004, 0.0004, 4.146380843321553e-28),
 (0.0003542308958920332, 0.0003531271885103128, 0.0011037073817204528),
 (0.00031886762200185933, 0.0003151902582904738, 0.0036773637113855376),
 (0.00029131852136167615, 0.00028440968709130996, 0.006908834270366193),
 (0.000269771700200215, 0.0002594214036544159, 0.010350296545799125),
 (0.00025285671202551, 0.00023912426106498125, 0.013732450960528761),
 (0.0002395368861190077, 0.0002226286145794821, 0.016908271539525583),
 (0.0002290218879809698, 0.0002092152054733013, 0.019806682507668486),
 (0.000220704767216189, 0.00019830213327707782, 0.02240263393911117),
 (0.0002141162189398047, 0.00018941833934687244, 0.024697879592932237),
 (0.0002088915166318055, 0.00018218221656230197, 0.02670930006950354),
 (0.00020474558646136793, 0.00017628455753430415, 0.028461028927063783),
 (0.00020145464285250956, 0.00017147471367688941, 0.029979929175620153),
 (0.0001988424231316343, 0.00016754938994130935, 0.03129303319032495),
 (0.00019676967120977001, 0.

Now we can optionally calculate some properties of the cell, e.g. its contractility and polarisation.

In [7]:
results = M.computeForceMoments(rmax=100e-6)

d:\repositories\saeno_py\saenopy\FiniteBodyForces.py:774: RuntimeWarning: invalid value encountered in true_divide
  contractility = np.sum(np.einsum("ki,ki->k", RR, f) / np.linalg.norm(RR, axis=1))
d:\repositories\saeno_py\saenopy\FiniteBodyForces.py:780: RuntimeWarning: invalid value encountered in true_divide
  eR = RR / np.linalg.norm(RR, axis=1)[:, None]
d:\repositories\saeno_py\saenopy\FiniteBodyForces.py:797: RuntimeWarning: invalid value encountered in true_divide
  vmid = vmid / np.linalg.norm(vmid)


In [8]:
# store the forces of the nodes
M.storeF("F.dat")
# store the positions and the displacements
M.storeRAndU("R.dat", "U.dat")
# store the center of each tetrahedron and a combined list with energies and volumina of the tetrahedrons
M.storeEandV("RR.dat", "EV.dat")

F.dat stored.
R.dat stored.
U.dat stored.
RR.dat stored.
EV.dat stored.


In [10]:
M.viewMesh(50, 1)